In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import spacy
nlp = spacy.load("de_core_news_lg")

In [3]:
df = pd.read_excel("../../data/df_de_final.xlsx")

print(f"Es sind {len(df)} Fragen vorhanden")

df.head()

Es sind 7459 Fragen vorhanden


,ID_Wahl,Datum,Frage_ID,Frage_Text,Frage_Typ,Bereich_ID,Bereich,ID_gesamt,Sprache
0,2.0,2019-10-20 00:00:00,5.0,"Wie beurteilen Sie diese Aussage: ""Wer sich ni...",options7,4734.0,Werthaltungen,Q00000,de
1,2.0,2019-10-20 00:00:00,8.0,"Wie beurteilen Sie diese Aussage: ""Die Bestraf...",options7,4734.0,Werthaltungen,Q00001,de
2,2.0,2019-10-20 00:00:00,11.0,"Wie beurteilen Sie diese Aussage: ""Für ein Kin...",options7,4734.0,Werthaltungen,Q00002,de
3,2.0,2019-10-20 00:00:00,14.0,Soll der Bund Ausländer/-innen bei der Integra...,options4,4231.0,Migration & Integration,Q00003,de
4,2.0,2019-10-20 00:00:00,17.0,Soll der Konsum von Cannabis legalisiert werden?,options4,4352.0,Gesellschaft & Ethik,Q00004,de


In [4]:
fragen = df["Frage_Text"].astype(str).tolist()


In [5]:
# spaCy-Vektorisierung
docs = [nlp(text) for text in tqdm(fragen, desc="Vektorisiere Fragen")]


Vektorisiere Fragen:   0%|          | 0/7459 [00:00<?, ?it/s]

In [6]:
print(f"{len(docs)} Fragen erfolgreich vektorisiert.")


7459 Fragen erfolgreich vektorisiert.


In [7]:
# Liste für Index und Text von Fragen ohne gültigen Vektor
leere_vektoren = []

for i, doc in enumerate(docs):
    if doc.vector_norm == 0:
        leere_vektoren.append((i, df.loc[i, "Frage_Text"]))


print(f"{len(leere_vektoren)} Fragen haben einen leeren Vektor.\n")


0 Fragen haben einen leeren Vektor.



In [8]:
from spacy.tokens import DocBin

# DocBin erstellen und alle docs hinzufügen
doc_bin = DocBin(store_user_data=True)  # speichert alle Infos, auch Vektoren
for doc in docs:
    doc_bin.add(doc)

# In Datei speichern
doc_bin.to_disk("docs_de.spacy")
print("Doc-Objekte gespeichert.")


Doc-Objekte gespeichert.


In [ ]:
# Code zum laden der docs

"""

from spacy.tokens import DocBin

# Modell wieder laden
nlp = spacy.load("de_core_news_lg")

# Datei laden
doc_bin = DocBin().from_disk("docs_de.spacy")

# Docs extrahieren
docs = list(doc_bin.get_docs(nlp.vocab))
print(f"{len(docs)} Docs erfolgreich geladen.")


"""

In [8]:
n = len(docs)

similarity_matrix = np.zeros((n, n))



for i in tqdm(range(n), desc="Berechne Ähnlichkeiten"):
    for j in range(i, n):  # nur obere Dreiecksmatrix + Diagonale
        sim = docs[i].similarity(docs[j])
        similarity_matrix[i, j] = sim
        similarity_matrix[j, i] = sim  # weil symmetrisch



Berechne Ähnlichkeiten:   0%|          | 0/7459 [00:00<?, ?it/s]

In [11]:
np.save("similarity_matrix_de.npy", similarity_matrix)

In [ ]:
# zum laden der similarity_matrix:

"""

similarity_matrix = np.load("similarity_matrix_de.npy")


"""